# <center>Régression linéaire</center>
<center>Prédiction du prix d'un vin</center>

*Objectifs* : 

Ce TP a pour but de mettre en oeuvre les méthodes relatives à la régression linéaire étudiées en cours afin de prédire le prix d'un vin. Plus particulièrement :
- Identifier les variables explicatives pertinentes.
- Identifier les dépendance non-linéaires entre réponse et prédicteurs.
- Identifier les interactions entre prédicteurs.
- Evaluer la qualité du modèle linéaire.
- Faire des prédictions à partir du modèle linéaire.

Dans cette séance, les librairies suivantes seront utilisées : installez-les si nécessaire et chargez-les

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Table des matières

<p><div class="lev1"><a href="#1-Introduction-:-présentation-du-jeu-de données-et-analyse-préliminaire"><span class="toc-item-num">1 - </span>Introduction : présentation du jeu de données et analyse préliminaire</a></div>
<p><div class="lev1"><a href="#2-Régression-linéaire-simple"><span class="toc-item-num">2 - </span>Régression linéaire simple</a></div>
<p><div class="lev1"><a href="#3-Régression-linéaire-multivariée"><span class="toc-item-num">3 - </span>Régression linéaire multivariée</a></div>
<p><div class="lev1"><a href="#4-Recherche-de-colinéarité"><span class="toc-item-num">4 - </span>Recherche de colinéarité</a></div>
<p><div class="lev1"><a href="#5-Terme-d'interaction"><span class="toc-item-num">5 - </span>Terme d'interaction</a></div>
<p><div class="lev1"><a href="#6-Représentation-diagnostic-et-intervalles-de-confiance"><span class="toc-item-num">6 - </span>Représentation diagnostic et intervalles de confiance</a></div>
<p><div class="lev1"><a href="#7-Nouvelles-prédictions"><span class="toc-item-num">7 - </span>Nouvelles prédictions</a></div>
<p><div class="lev1"><a href="#Annexe"><span class="toc-item-num"></span>Annexe</a></div>

## 1 Introduction : présentation du jeu de données et analyse préliminaire

Ce problème reprend des données et un modèle ayant donné lieu à une intense polémique : en mars
1990, Orley Ashenfelter, un professeur d’économie de Princeton publie un article affirmant que l’on peut
prédire la qualité du vin, et son prix, sans même le goûter, à partir d'une formule mathématique. Scandale
chez les experts de l’œnologie : “Ashenfelter is an absolute total sham” ou “rather like a movie critic who
never goes to see the movie but tells you how good it is based on the actors and the director”. Dans les
faits, la “Bordeaux-formula” donne des résultats intéressants qui parfois dépassent les prévisions des
experts. L’approche analytique est maintenant lentement adoptée par la communauté. 

Ces données proviennent du site de [Ashenfelter, Liquid Assets](http://liquidasset.com/). Elles comprennent $25$ observations et $7$ variables.

|Nom  |Description|
|-----|-----------|
|Year  |Année de production du vin |
|Price|prix de vente (une approximation de la qualité du vin) en 1990|
|WinterRain|Précipitations pendant l’hiver|
| AGST|Average Growing Season Temperature|
|      |Température moyenne pendant la croissance|
|HarvestRain|Précipitations pendant l'été|
|Age|âge du vin (en général les vins les plus anciens sont plus chers)|
|FrancePop|Population française|

La variable dépendante est 'Price'.

Les variables indépendantes sont 'Year', 'WinterRain', 'AGST', 'HarvestRain', 'Age', 'FrancePop'.

><u>Tâche 1</u>. Chargez les données. Examinez quelles sont les caractéristiques des variables du jeu de données.

><u>Tâche 2</u>. Regardez la corrélation entre :
>- 'AGST' et 'PRICE' d'une part et entre 'AGST' et log('PRICE') d'autre part,
>- 'HarvestRain' et 'PRICE' d'une part et entre 'HarvestRain' et log('PRICE') d'autre part.
>
> A l'aide de la fonction <span style="border:1px; border-style:solid">skew</span>, regardez le coefficent d'asymétrie de 'Price'.

Dans la suite, nous utiliserons donc la transformation logarithmique de la variable à prédire.

><u>Tâche 3</u>. Tracer le logarithme du prix de vente en fonction de
'AGST' d'une part puis de 'HarvestRain'. Qu'en déduisez-vous sur les relations entre ces variables ?

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 2 Régression linéaire simple

><u>Tâche 4</u>. Quel est le modèle *baseline* par défaut (le modèle prédictif construit sans aucune variables explicatives) ? On l'appelle <span style="border:1px; border-style:solid">model0</span>. 
>
>Effectuer une première régression <span style="border:1px; border-style:solid">model1</span> du (logarithme du) prix en fonction de AGST. Utiliser pour cela la fonction <span style="border:1px; border-style:solid">LinearRegressor</span> se trouvant dans <u>sklearn.linear_model</u>. 
>    
> Calculer le RSS pour ce modèle et comparez le au RSS du modèle baseline (il s'agit alors du TSS). 
>
>Déduisez-en le $R^2$ du modèle, et comparez
le à celui qui est fourni par la fonction <span style="border:1px; border-style:solid">r2_score</span> qui se trouve dans <u>sklearn.metrics</u>.
>
> Calculer le residual standard eror (RSE) des deux modèles.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 3 Régression linéaire multivariée

><u>Tâche 5</u>. Ajouter la nouvelle variable indépendante 'HarvestRain' pour un nouveau modèle <span style="border:1px; border-style:solid">model2</span>. Considérez la nouvelle valeur du $R^2$ : le modèle est il amélioré, cette variable est-elle significative ? Donner à nouveau la valeur du RSE.

><u>Tâche 6</u>. Construire maintenant un modèle avec l'ensemble des variables disponibles. Donnez le $R^2$ et le RSE correspondants.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 4 Recherche de colinéarité

><u>Tâche 7</u>. Les colinarités peuvent impacter négativement les performances du modèle. Repérez-les à l'aide des coefficients de corrélations, des nuages de points et par le raisonnement. Créer un nouveau modèle linéaire sans colinéarité et évaluez-le. 

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 5 Terme d'interaction

><u>Tâche 8</u>. Il est raisonnable de penser que la qualité d'un vin, est donc son prix, peut ếtre impactée par les précipitations hivernales et estivales conjointement. Ajouter au jeu de données la variable 'WinterRain_HarvestRain', produit des variables 'WinterRain' et 'HarvestRain'. Créer le modèle linéaire incluant les variables précédemment sélectionnées et la nouvelle variable d'interaction. Le RSE et le $R^2$ en sont-ils améliorés ?

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 6 Représentation diagnostic et intervalles de confiance

><u>Tâche 9 : test de l'hypothèse d'homoscédasticité</u>.  Tracer le *residual plot*, c'est-à-dire les résidus obtenus en fonction des valeurs prédites et conclure quant à l'homoscédasticité du modèle.

><u>Tâche 10</u>. Afficher l'estimation de l'intercept et des coefficients du modèle linéaire. Donner leur intervalles de confiance à $95$ %.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## 7 Nouvelles prédictions

><u>Tâche 11</u>. Il s'agit enfin de prédire le prix de vente pour les années 1979 et 1980. Ces données sont
fournies dans le fichier wine_test.csv. Effectez la prédiction. Comparez ces deux résultats aux valeurs exactes.

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>

## Annexe

Il existe d'autre librairies de régression linéaire en python, avec des méthodes statistiques plus riches.

In [99]:
import statsmodels.api as sm

In [1]:
#mod = sm.OLS(y,X)
#res = mod.fit()
#print(res.summary())
#print(res.conf_int(0.05))   # 95% confidence interval

<div class="lev1"><a href="#Table-des-matières"><span class="toc-item-num"></span>Retour à la table des matières</a></div>